# fileorg

> Fill in a module description here

In [ ]:
#| default_exp fileorg

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import shutil
from pathlib import Path

from fastai.vision.all import get_image_files

In [ ]:
#| export
DEFAULT_HOME_DIR=Path.home()/'singleline_data'

def _path_from_env(variable, default):
    value = os.environ.get(variable)
    if value and os.path.isabs(value):
        print(f'found env var {variable}, using: {value}')
        return Path(value)
    return Path(default)

def singleline_data_home(default=DEFAULT_HOME_DIR):
    return _path_from_env('SINGLELINE_DATA_HOME', default)

def raster_dir(default=DEFAULT_HOME_DIR):
    return singleline_data_home(default) / 'raster'

def svg_dir(default=DEFAULT_HOME_DIR):
    return singleline_data_home(default) / 'svg'

def stroke3_dir(default=DEFAULT_HOME_DIR):
    return singleline_data_home(default) / 'stroke3'

def epoch_subdir(base: Path, epoch: str):
    return base / f"epoch-{epoch}"


In [ ]:
#| export
def flat_sketchbook_paths(data_root, exclude_parents=["xtra", "priv"]):
    for p in get_image_files(data_root):
        # ex. '016.jpg'
        name = os.path.basename(p)
        # ex. 'art'
        parent_dir = os.path.dirname(p)
        parent_name = os.path.basename(parent_dir)
        if parent_name in exclude_parents:
            continue
        # ex. 'sb44'
        grandparent_dir = os.path.dirname(parent_dir)
        grandparent_name = os.path.basename(grandparent_dir)

        # # should be same as data_root
        # greatgrandparent_dir = os.path.dirname(grandparent_dir)
        # print(greatgrandparent_dir, data_root)
        # assert data_root == str(greatgrandparent_dir)

        dest_fname = f"{parent_name}/{grandparent_name}p{name}"

        yield {
            "source_abs": p,
            "dest_fname": dest_fname,
            "parent_name": parent_name,
            "grandparent_name": grandparent_name,
        }

In [ ]:
#| export
def flatten_sketchbooks(source_root, dest_root):
    parent_counts = {}
    grandparent_counts = {}
    for f in flat_sketchbook_paths(source_root):
        source_abs = f["source_abs"]
        dest_abs = Path(dest_root) / f["dest_fname"]
        dest_dir = os.path.dirname(dest_abs)
        if not os.path.isdir(dest_dir):
            print(f"creating {dest_dir}")
            os.makedirs(dest_dir)
        if not os.path.exists(dest_abs):
            print(f"copying {source_abs} to {dest_abs}")
            shutil.copy(source_abs, dest_abs)

        parent_cnt = parent_counts.get(f["parent_name"], 0)
        parent_counts[f["parent_name"]] = parent_cnt + 1
        grandparent_cnt = grandparent_counts.get(f["grandparent_name"], 0)
        grandparent_counts[f["grandparent_name"]] = grandparent_cnt + 1
    return parent_counts, grandparent_counts

In [ ]:
# source_root = "../data/00_RAW/sample"
# dest_root = "../data/01_FLAT/sample"

# # source_root = "/Users/al/Dropbox/2-Areas/2-Sketchbooks/sketchbooks"
# # dest_root = "/Users/al/Dropbox/2-Areas/2-Sketchbooks/raster/epoch-20231214/01_FLAT"

# parents, grandparents = flatten_sketchbooks(source_root, dest_root)

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()